# global imports

In [1]:
import numpy as np
from random import *
import math

# define classes

In [2]:
class Neuron:
    
    def __init__(self):
        self.w=[] 
        self.b=0
        self.o=-1
        self.errorDelta=[]
        
    def activation(self, x):
        # using tanh
        return 2.0 / (1.0 + math.exp(-2*x)) - 1.
    
    def derivative_activation(self,x):
        return 1 - x ** 2
    
    def setInputDimension(self,n_inputs):
        self.w=np.random.rand(n_inputs)
        #self.w=np.array([-0.5] * n_inputs)
        
    def updateWeights(self, lr, inputSignal):
        self.w=self.w + np.multiply(self.errorDelta,inputSignal)*lr
        self.b=self.b + self.errorDelta * lr
        
    def elaborate(self,inputSignal):
        value=np.dot(self.w,inputSignal)+self.b
        self.o=self.activation(value)
        return self.o
    
    def elaborateErrorDelta(self, error):
        self.errorDelta=error * self.derivative_activation(self.o)
        return self.errorDelta
    
class DenseLayer:
    
    def __init__(self,n_neurons,dim_input):
        self.neurons=[]
        self.outputs=[]
        for i in range(n_neurons):
            neuron=Neuron()
            neuron.setInputDimension(dim_input)
            self.addNeuron(neuron)
            self.outputs.append(neuron.o)
        
    def addNeuron(self,neuron):
        self.neurons.append(neuron)
        
    def elaborate(self, inputSignal):
        for i in range(len(self.neurons)):
            self.outputs[i]=self.neurons[i].elaborate(inputSignal)
        return self.outputs
    
    def updateWeights(self, lr, inputSignal):
        for neuron in self.neurons:
            neuron.updateWeights(lr, inputSignal)            
            
            
    def backpropagate(self, nextLayer=None, expected=None):
        if (nextLayer is None):
            for i in range(len(self.neurons)):
                error=expected-self.neurons[i].o
                self.neurons[i].elaborateErrorDelta(error)
        else:
            for i in range(len(self.neurons)):
                error=0.
                for j in range(len(nextLayer.neurons)):
                    error+=nextLayer.neurons[j].errorDelta 
                self.neurons[i].elaborateErrorDelta(error)


class Network:
    
    def __init__(self):
        self.layers=[]
        
    def addLayer(self,layer):
        self.layers.append(layer)
        
    def predict(self, inputSignal):
        for i in range(len(self.layers)):
            if ( i==0 ):
                self.layers[i].elaborate(inputSignal)
            else:
                self.layers[i].elaborate(self.layers[i-1].outputs)
        return self.layers[-1].outputs
    
    def backpropagate(self, expected):
        for i in reversed(range(len(self.layers))):
            if (i==len(self.layers)-1):
                self.layers[i].backpropagate(None, expected)
            else:
                self.layers[i].backpropagate(self.layers[i+1])
                
    def updateWeights(self, lr, inputSignal):
        for i in range(len(self.layers)):
            if ( i==0 ):
                self.layers[i].updateWeights(lr, inputSignal)
            else:
                self.layers[i].updateWeights(lr, np.array(self.layers[i-1].outputs))
                
    def toString(self):
        for i in range(len(self.layers)):
            print("layer",i)
            for j in range(len(self.layers[i].neurons)):
                neuron=self.layers[i].neurons[j]
                print("----------------------")
                print("neuron",j)
                print("weights: ")
                print("\t",neuron.w[:-1])
                print("\t",neuron.w[-1])
                print("----------------------")
            print("##########################")
            
    def train(self, X, Y, epochs, lr):
        for e in range(epochs):
            print("epoch ",e," of ",epochs)
            err=0
            for i in range(len(X)):
                pred=self.predict(X[i])[0]
                self.backpropagate(Y[i])
                self.updateWeights(lr, X[i])
                err+=abs(Y[i]-pred)
            print("avg error: ",err/len(X))
                
        
    
    

# test

In [3]:
INPUT_SIZE=2
OUTPUT_SIZE=1
HIDDEN_SIZE=4

LOW=-1.
HIGH=1.

INPUT=np.array([LOW,LOW])

X_TRAIN=np.array([
    [LOW,HIGH],
    [HIGH,LOW],
    [HIGH,HIGH],
    [LOW,LOW]
])

Y_TRAIN=np.array([
    [LOW],
    [LOW],
    [HIGH],
    [HIGH]
])

hiddenLayer=DenseLayer(HIDDEN_SIZE,INPUT_SIZE)
outputLayer=DenseLayer(OUTPUT_SIZE, HIDDEN_SIZE)

network=Network()

network.addLayer(hiddenLayer)
network.addLayer(outputLayer)

network.predict(INPUT)



[-0.7785041252359798]

In [4]:
network.train( X_TRAIN, Y_TRAIN, epochs=5000, lr=0.01)

epoch  0  of  5000
avg error:  [1.0107983]
epoch  1  of  5000
avg error:  [0.99967974]
epoch  2  of  5000
avg error:  [0.98946339]
epoch  3  of  5000
avg error:  [0.98016879]
epoch  4  of  5000
avg error:  [0.97178649]
epoch  5  of  5000
avg error:  [0.96428134]
epoch  6  of  5000
avg error:  [0.95759833]
epoch  7  of  5000
avg error:  [0.95166922]
epoch  8  of  5000
avg error:  [0.94641881]
epoch  9  of  5000
avg error:  [0.94177003]
epoch  10  of  5000
avg error:  [0.93764778]
epoch  11  of  5000
avg error:  [0.93398151]
epoch  12  of  5000
avg error:  [0.93070668]
epoch  13  of  5000
avg error:  [0.92776548]
epoch  14  of  5000
avg error:  [0.92510692]
epoch  15  of  5000
avg error:  [0.92268662]
epoch  16  of  5000
avg error:  [0.9204663]
epoch  17  of  5000
avg error:  [0.91841317]
epoch  18  of  5000
avg error:  [0.91649937]
epoch  19  of  5000
avg error:  [0.91470126]
epoch  20  of  5000
avg error:  [0.9129989]
epoch  21  of  5000
avg error:  [0.9113755]
epoch  22  of  5000
avg 

epoch  250  of  5000
avg error:  [0.68483866]
epoch  251  of  5000
avg error:  [0.68447879]
epoch  252  of  5000
avg error:  [0.68412226]
epoch  253  of  5000
avg error:  [0.68376902]
epoch  254  of  5000
avg error:  [0.68341907]
epoch  255  of  5000
avg error:  [0.68307236]
epoch  256  of  5000
avg error:  [0.68272887]
epoch  257  of  5000
avg error:  [0.68238859]
epoch  258  of  5000
avg error:  [0.68205147]
epoch  259  of  5000
avg error:  [0.6817175]
epoch  260  of  5000
avg error:  [0.68138664]
epoch  261  of  5000
avg error:  [0.68105889]
epoch  262  of  5000
avg error:  [0.68073421]
epoch  263  of  5000
avg error:  [0.68041257]
epoch  264  of  5000
avg error:  [0.68009396]
epoch  265  of  5000
avg error:  [0.67977835]
epoch  266  of  5000
avg error:  [0.67946571]
epoch  267  of  5000
avg error:  [0.67915603]
epoch  268  of  5000
avg error:  [0.67884929]
epoch  269  of  5000
avg error:  [0.67854545]
epoch  270  of  5000
avg error:  [0.6782445]
epoch  271  of  5000
avg error:  [0.

epoch  505  of  5000
avg error:  [0.66746934]
epoch  506  of  5000
avg error:  [0.66770188]
epoch  507  of  5000
avg error:  [0.66793821]
epoch  508  of  5000
avg error:  [0.66817837]
epoch  509  of  5000
avg error:  [0.66842242]
epoch  510  of  5000
avg error:  [0.6686704]
epoch  511  of  5000
avg error:  [0.66892236]
epoch  512  of  5000
avg error:  [0.66917835]
epoch  513  of  5000
avg error:  [0.66943842]
epoch  514  of  5000
avg error:  [0.66970262]
epoch  515  of  5000
avg error:  [0.66997099]
epoch  516  of  5000
avg error:  [0.67024361]
epoch  517  of  5000
avg error:  [0.67052051]
epoch  518  of  5000
avg error:  [0.67080176]
epoch  519  of  5000
avg error:  [0.67108741]
epoch  520  of  5000
avg error:  [0.67137752]
epoch  521  of  5000
avg error:  [0.67167215]
epoch  522  of  5000
avg error:  [0.67197135]
epoch  523  of  5000
avg error:  [0.67227519]
epoch  524  of  5000
avg error:  [0.67258373]
epoch  525  of  5000
avg error:  [0.67289704]
epoch  526  of  5000
avg error:  [0

epoch  763  of  5000
avg error:  [0.57098889]
epoch  764  of  5000
avg error:  [0.56813656]
epoch  765  of  5000
avg error:  [0.5653027]
epoch  766  of  5000
avg error:  [0.56248743]
epoch  767  of  5000
avg error:  [0.55969086]
epoch  768  of  5000
avg error:  [0.55691307]
epoch  769  of  5000
avg error:  [0.55415416]
epoch  770  of  5000
avg error:  [0.55141417]
epoch  771  of  5000
avg error:  [0.54869316]
epoch  772  of  5000
avg error:  [0.54599116]
epoch  773  of  5000
avg error:  [0.54330821]
epoch  774  of  5000
avg error:  [0.54064432]
epoch  775  of  5000
avg error:  [0.53799949]
epoch  776  of  5000
avg error:  [0.53537373]
epoch  777  of  5000
avg error:  [0.53276701]
epoch  778  of  5000
avg error:  [0.53017934]
epoch  779  of  5000
avg error:  [0.52761067]
epoch  780  of  5000
avg error:  [0.52506099]
epoch  781  of  5000
avg error:  [0.52253024]
epoch  782  of  5000
avg error:  [0.52001838]
epoch  783  of  5000
avg error:  [0.51752537]
epoch  784  of  5000
avg error:  [0

avg error:  [0.25160019]
epoch  1005  of  5000
avg error:  [0.25106851]
epoch  1006  of  5000
avg error:  [0.25053961]
epoch  1007  of  5000
avg error:  [0.25001347]
epoch  1008  of  5000
avg error:  [0.24949007]
epoch  1009  of  5000
avg error:  [0.24896939]
epoch  1010  of  5000
avg error:  [0.24845141]
epoch  1011  of  5000
avg error:  [0.2479361]
epoch  1012  of  5000
avg error:  [0.24742345]
epoch  1013  of  5000
avg error:  [0.24691343]
epoch  1014  of  5000
avg error:  [0.24640603]
epoch  1015  of  5000
avg error:  [0.24590122]
epoch  1016  of  5000
avg error:  [0.24539899]
epoch  1017  of  5000
avg error:  [0.24489931]
epoch  1018  of  5000
avg error:  [0.24440217]
epoch  1019  of  5000
avg error:  [0.24390754]
epoch  1020  of  5000
avg error:  [0.24341542]
epoch  1021  of  5000
avg error:  [0.24292576]
epoch  1022  of  5000
avg error:  [0.24243857]
epoch  1023  of  5000
avg error:  [0.24195382]
epoch  1024  of  5000
avg error:  [0.24147149]
epoch  1025  of  5000
avg error:  [0

...

epoch  3657  of  5000
avg error:  [0.0623888]
epoch  3658  of  5000
avg error:  [0.0623768]
epoch  3659  of  5000
avg error:  [0.06236481]
epoch  3660  of  5000
avg error:  [0.06235282]
epoch  3661  of  5000
avg error:  [0.06234084]
epoch  3662  of  5000
avg error:  [0.06232887]
epoch  3663  of  5000
avg error:  [0.0623169]
epoch  3664  of  5000
avg error:  [0.06230494]
epoch  3665  of  5000
avg error:  [0.06229299]
epoch  3666  of  5000
avg error:  [0.06228104]
epoch  3667  of  5000
avg error:  [0.0622691]
epoch  3668  of  5000
avg error:  [0.06225717]
epoch  3669  of  5000
avg error:  [0.06224524]
epoch  3670  of  5000
avg error:  [0.06223332]
epoch  3671  of  5000
avg error:  [0.0622214]
epoch  3672  of  5000
avg error:  [0.0622095]
epoch  3673  of  5000
avg error:  [0.06219759]
epoch  3674  of  5000
avg error:  [0.0621857]
epoch  3675  of  5000
avg error:  [0.06217381]
epoch  3676  of  5000
avg error:  [0.06216193]
epoch  3677  of  5000
avg error:  [0.06215005]
epoch  3678  of  500

epoch  4079  of  5000
avg error:  [0.05784517]
epoch  4080  of  5000
avg error:  [0.05783551]
epoch  4081  of  5000
avg error:  [0.05782585]
epoch  4082  of  5000
avg error:  [0.05781619]
epoch  4083  of  5000
avg error:  [0.05780654]
epoch  4084  of  5000
avg error:  [0.05779689]
epoch  4085  of  5000
avg error:  [0.05778725]
epoch  4086  of  5000
avg error:  [0.05777761]
epoch  4087  of  5000
avg error:  [0.05776797]
epoch  4088  of  5000
avg error:  [0.05775834]
epoch  4089  of  5000
avg error:  [0.05774872]
epoch  4090  of  5000
avg error:  [0.0577391]
epoch  4091  of  5000
avg error:  [0.05772948]
epoch  4092  of  5000
avg error:  [0.05771987]
epoch  4093  of  5000
avg error:  [0.05771027]
epoch  4094  of  5000
avg error:  [0.05770066]
epoch  4095  of  5000
avg error:  [0.05769107]
epoch  4096  of  5000
avg error:  [0.05768147]
epoch  4097  of  5000
avg error:  [0.05767189]
epoch  4098  of  5000
avg error:  [0.0576623]
epoch  4099  of  5000
avg error:  [0.05765272]
epoch  4100  of

epoch  4329  of  5000
avg error:  [0.0555638]
epoch  4330  of  5000
avg error:  [0.05555519]
epoch  4331  of  5000
avg error:  [0.05554658]
epoch  4332  of  5000
avg error:  [0.05553797]
epoch  4333  of  5000
avg error:  [0.05552937]
epoch  4334  of  5000
avg error:  [0.05552077]
epoch  4335  of  5000
avg error:  [0.05551218]
epoch  4336  of  5000
avg error:  [0.05550358]
epoch  4337  of  5000
avg error:  [0.055495]
epoch  4338  of  5000
avg error:  [0.05548641]
epoch  4339  of  5000
avg error:  [0.05547784]
epoch  4340  of  5000
avg error:  [0.05546926]
epoch  4341  of  5000
avg error:  [0.05546069]
epoch  4342  of  5000
avg error:  [0.05545212]
epoch  4343  of  5000
avg error:  [0.05544356]
epoch  4344  of  5000
avg error:  [0.055435]
epoch  4345  of  5000
avg error:  [0.05542644]
epoch  4346  of  5000
avg error:  [0.05541789]
epoch  4347  of  5000
avg error:  [0.05540934]
epoch  4348  of  5000
avg error:  [0.05540079]
epoch  4349  of  5000
avg error:  [0.05539225]
epoch  4350  of  5

epoch  4577  of  5000
avg error:  [0.05353828]
epoch  4578  of  5000
avg error:  [0.05353053]
epoch  4579  of  5000
avg error:  [0.05352279]
epoch  4580  of  5000
avg error:  [0.05351506]
epoch  4581  of  5000
avg error:  [0.05350732]
epoch  4582  of  5000
avg error:  [0.05349959]
epoch  4583  of  5000
avg error:  [0.05349187]
epoch  4584  of  5000
avg error:  [0.05348414]
epoch  4585  of  5000
avg error:  [0.05347642]
epoch  4586  of  5000
avg error:  [0.05346871]
epoch  4587  of  5000
avg error:  [0.05346099]
epoch  4588  of  5000
avg error:  [0.05345328]
epoch  4589  of  5000
avg error:  [0.05344557]
epoch  4590  of  5000
avg error:  [0.05343787]
epoch  4591  of  5000
avg error:  [0.05343017]
epoch  4592  of  5000
avg error:  [0.05342247]
epoch  4593  of  5000
avg error:  [0.05341477]
epoch  4594  of  5000
avg error:  [0.05340708]
epoch  4595  of  5000
avg error:  [0.05339939]
epoch  4596  of  5000
avg error:  [0.05339171]
epoch  4597  of  5000
avg error:  [0.05338402]
epoch  4598  

epoch  4773  of  5000
avg error:  [0.05207948]
epoch  4774  of  5000
avg error:  [0.05207233]
epoch  4775  of  5000
avg error:  [0.05206518]
epoch  4776  of  5000
avg error:  [0.05205804]
epoch  4777  of  5000
avg error:  [0.05205089]
epoch  4778  of  5000
avg error:  [0.05204375]
epoch  4779  of  5000
avg error:  [0.05203661]
epoch  4780  of  5000
avg error:  [0.05202948]
epoch  4781  of  5000
avg error:  [0.05202234]
epoch  4782  of  5000
avg error:  [0.05201521]
epoch  4783  of  5000
avg error:  [0.05200809]
epoch  4784  of  5000
avg error:  [0.05200096]
epoch  4785  of  5000
avg error:  [0.05199384]
epoch  4786  of  5000
avg error:  [0.05198672]
epoch  4787  of  5000
avg error:  [0.05197961]
epoch  4788  of  5000
avg error:  [0.05197249]
epoch  4789  of  5000
avg error:  [0.05196538]
epoch  4790  of  5000
avg error:  [0.05195828]
epoch  4791  of  5000
avg error:  [0.05195117]
epoch  4792  of  5000
avg error:  [0.05194407]
epoch  4793  of  5000
avg error:  [0.05193697]
epoch  4794  

In [5]:
network.predict(np.array([HIGH,HIGH]))

[0.9469687130233575]

In [6]:
network.predict(np.array([LOW,LOW]))

[0.9349788257416485]

In [7]:
network.predict(np.array([LOW,HIGH]))

[-0.9579956792362905]

In [8]:
network.predict(np.array([HIGH,LOW]))

[-0.9579893527595497]